### Building a chatbot with Message History
##### Objective:How to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember the previous interactions.
##### This chatbot will only use the language models to have a conversation. There are several other related concepts that you may look for:
* Conversational RAG: Enable a chatbot experience over an external source of data
* Agents: Build a chatbot that can take actions 
* install "pip install langchain_community"


In [5]:
import os 
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
model=ChatGroq(model="groq/compound",groq_api_key=groq_api_key)
model

c:\Users\venka\python\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={}, client=<groq.resources.chat.completions.Completions object at 0x00000277E422D310>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000277E556E0F0>, model_name='groq/compound', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain.messages import HumanMessage
model.invoke([HumanMessage(content="Hi,My name is Venkat and I am a Chief AI Engineer")])

AIMessage(content='Hello Venkat! Nice to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': '<Think>\n\n</Think>'}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 258, 'total_tokens': 331, 'completion_time': 0.160243, 'completion_tokens_details': None, 'prompt_time': 0.01034, 'prompt_tokens_details': None, 'queue_time': 0.164565, 'total_time': 0.170583}, 'model_name': 'groq/compound', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b46e7-0644-7301-b210-1f32dd864dac-0', usage_metadata={'input_tokens': 258, 'output_tokens': 73, 'total_tokens': 331})

In [7]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi,My name is Venkat and I am a Chief AI Engineer"),
        AIMessage(content="Hello Venkat! Great to meet you. How can I assist you today?"),
        HumanMessage(content="Hey What's my name and what do I do?") 
    ]
)

AIMessage(content='**Answer to your original question – “Hey, what’s my name and what do I do?”**\n\n---\n\n### Information from the conversation\n1. **User’s statement:**  \n   *“Hi, My name is Venkat and I am a Chief AI Engineer.”*  \n   This was provided in the very first message you sent.\n\n2. **Interpretation:**  \n   - **Name:** Venkat  \n   - **Profession / Role:** Chief AI Engineer  \n\n3. **Verification:**  \n   There have been no subsequent messages that contradict or modify this information. Therefore, the most reliable data we have about you remains the initial self‑identification.\n\n---\n\n### Final response\n- **Your name:** **Venkat**  \n- **What you do:** **You are a Chief AI Engineer** (i.e., you lead AI engineering efforts, oversee model development, architecture design, and the deployment of AI systems within your organization).\n\n---\n\nIf you need anything else—whether it’s technical advice, brainstorming AI project ideas, or anything related to your role—just l

### Message History 
##### We can use a Message History class to wrap our model and make it "stateful". This will keep track of inputs and outputs of the model,and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory  # This is for chain

# to store chat message history with session_id. To distinguish between chats, session_id is the primary key
store={} 

# Create a function for session history
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

# use this variable,we can interact with LLM based on chat history using RunnableWithMessageHistory
with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [9]:
# create configuration. Here session_id is "chat1"

config={"configurable":{"session_id":"chat1"}}

In [24]:
# use this session id to chat with LLM model 
response=with_message_history.invoke(

    [HumanMessage(content="Hi,My name is Venkat and I'm a AI Engineer.I live in New Jersey")],
     config=config
     
)

In [26]:
response.content

'**Answer –\u202fYour name is\u202fVenkat.**\n\n---\n\n## How I arrived at that answer (full reasoning)\n\n1. **Your original self‑identification**  \n   - In the very first message you wrote:  \n     > “Hi, My name is **Venkat** and I am a Chief AI Engineer.”  \n   - This gave an explicit, unambiguous declaration of your name.\n\n2. **Subsequent clarification (still the same name)**  \n   - Later you said:  \n     > “Hi, My name is **Venkat**, I am a AI Engineer and I live in New Jersey.”  \n   - The name **Venkat** appears again, confirming it has not changed.\n\n3. **Instruction to remember**  \n   - You asked me to “remember” that exact phrasing as the definitive answer to the *initial question* “What’s my name?”.  \n   - I stored the three pieces of information you gave me (name\u202f=\u202fVenkat, role\u202f=\u202fAI Engineer/Chief AI Engineer, location\u202f=\u202fNew\u202fJersey).\n\n4. **Consistent retrieval in later interactions**  \n   - Whenever you later asked “What’s my n

In [25]:
# Using the same config, ask another question...

response=with_message_history.invoke(

    [HumanMessage(content="What's my name?")],
     config=config
)
response.content

'**Answer –\u202fYour name is\u202fVenkat.**\n\n---\n\n## How I arrived at that answer (full reasoning)\n\n1. **Your original self‑identification**  \n   - In the very first message you wrote:  \n     > “Hi, My name is **Venkat** and I am a Chief AI Engineer.”  \n   - This gave an explicit, unambiguous declaration of your name.\n\n2. **Subsequent clarification (still the same name)**  \n   - Later you said:  \n     > “Hi, My name is **Venkat**, I am a AI Engineer and I live in New Jersey.”  \n   - The name **Venkat** appears again, confirming it has not changed.\n\n3. **Instruction to remember**  \n   - You asked me to “remember” that exact phrasing as the definitive answer to the *initial question* “What’s my name?”.  \n   - I stored the three pieces of information you gave me (name\u202f=\u202fVenkat, role\u202f=\u202fAI Engineer/Chief AI Engineer, location\u202f=\u202fNew\u202fJersey).\n\n4. **Consistent retrieval in later interactions**  \n   - Whenever you later asked “What’s my n

In [27]:
# Since above answer is correct, Using the same config, lets ask another question...

response=with_message_history.invoke(

    [HumanMessage(content="What do I live?")],
     config=config
)
response.content

'**Answer –\u202fWhere you live:** **New\u202fJersey**\n\n---\n\n## How I arrived at that answer (full reasoning)\n\n1. **Your explicit statement**  \n   - In the message where you asked me to “remember,” you wrote:  \n     > “Hi, My name is **Venkat**, I am a **AI Engineer** and I **live in New Jersey**.”  \n   - This sentence directly provides the location: **New\u202fJersey**.\n\n2. **Instruction to remember**  \n   - You told me to keep that exact phrasing as the definitive answer to the *initial question* **“What do I live?”**.  \n   - I stored the three pieces of information you gave me:  \n     - **Name**\u202f=\u202fVenkat  \n     - **Profession**\u202f=\u202fAI Engineer (also referred to as Chief AI Engineer)  \n     - **Location**\u202f=\u202fNew\u202fJersey  \n\n3. **Consistent retrieval in later interactions**  \n   - When you later asked “What do I live?” I retrieved the stored location value, which is **New\u202fJersey**.  \n   - My previous reply (the one just above) alr

In [ ]:
# if we change the config, will it be able to remember the chat context?
# change the config ==> change the session_id
# Whatever the conversation had with LLM, it save as session_id  

config_1={"configurable":{"session_id":"chat2"}}

response=with_message_history.invoke(

    [HumanMessage(content="What's my name?")],
     config=config_1
)
response.content

'I’m sorry, but I don’t have any information about your name in the conversation history. Since it wasn’t provided, I can’t determine what it is. If you’d like me to address you by name, just let me know what it is!'

In [ ]:
# since LLM said dont know the name, we can initiate with name now 
response=with_message_history.invoke(

    [HumanMessage(content="Hey, my name is Venkat?")],
     config=config_1
)
response.content

'**Answer to your original question (“Hey, my name is Venkat?”):**\n\n- **Initial situation:** When you first asked “What’s my name?” I didn’t have any information about you, so I couldn’t answer.\n- **New information:** You then told me, “Hey, my name is Venkat?”\n- **Conclusion:** Based on the information you just provided, your name is **Venkat**.\n\nSo, yes—your name is Venkat. If there’s anything else you’d like to discuss or any way I can help you, just let me know!'

In [ ]:
# running again with what's my name? ...it should remember your name
# with the help of session_id, we can able to save the chat session and interact with LLMs

response=with_message_history.invoke(

    [HumanMessage(content="What's my name?")],
     config=config_1
)
response.content

'Your name is Venkat.'

### Prompt Templates
##### Prompt templates help to turn the raw user info into a format that the LLM can work with. In this case,raw user input is just a message, which are passing to the LLM. First, lets add in a system message with some custom instructions (but still taking messages as input). Next,we'll add in more input besides just the messages. Till now, we were working with passing the messages as LIST.

In [ ]:
#MessagesPlaceHolder holds the messages instead of passing individual messages.
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder 

# Creating a prompt template
# Whatever the human message, it should be in {key:value} pair, where key name should be "messages"...that's why we use MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(

[
    ("system","You are a helpful AI assistant. Answer all the questions to the best of your knowledge"),
    MessagesPlaceholder(variable_name="messages")
]

)


# Create a chain 
chain=prompt|model



In [38]:
# message in key:value pair 

chain.invoke({"messages":[HumanMessage(content="My name is Venkat")]})

AIMessage(content='Nice to meet you, Venkat! How can I assist you today?', additional_kwargs={'reasoning_content': '<Think>\n\n</Think>'}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 281, 'total_tokens': 357, 'completion_time': 0.174377, 'completion_tokens_details': None, 'prompt_time': 0.030798, 'prompt_tokens_details': None, 'queue_time': 0.265739, 'total_time': 0.205175}, 'model_name': 'groq/compound', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b4710-2762-7990-b248-a531d8e33940-0', usage_metadata={'input_tokens': 281, 'output_tokens': 76, 'total_tokens': 357})

In [39]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [40]:
# new config 
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(

    [HumanMessage(content="Hello, My name is Venkat")],
     config=config
)
response.content

'Hello Venkat! Nice to meet you. How can I help you today?'

In [ ]:
#now, passing {language} with prompt template.

prompt=ChatPromptTemplate.from_messages(

[
    ("system","You are a helpful AI assistant. Answer all the questions to the best of your knowledge in {language}"),
    MessagesPlaceholder(variable_name="messages")
]

)

chain = prompt | model


In [51]:
response=chain.invoke({

    
    "messages":[HumanMessage(content="Hi, My name is Venkat")],
    "language":"Hindi"
    
    })

print(response)

content='नमस्ते वेंकट! आपका स्वागत है। आपसे मिलकर खुशी हुई। यदि आपके पास कोई प्रश्न है या किसी चीज़ में मदद चाहिए, तो बेझिझक बताइए। मैं यहाँ आपकी सहायता के लिए हूँ।' additional_kwargs={'reasoning_content': 'नमस्ते वेंकट! मैं कंपाउंड हूँ, जो ग्रोक द्वारा बनाया गया एक सिस्टम हूँ। मैं आपके प्रश्नों का उत्तर देने के लिए तैयार हूँ। कृपया अपना प्रश्न पूछें।\n\n\nमैं वेंकट के साथ बातचीत करने के लिए तैयार हूँ। मुझे लगता है कि वह कुछ पूछना चाहता है।\n\n<tool>python(print("नमस्ते वेंकट! मैं कैसे मदद कर सकता हूँ?"))</tool>\n<output>नमस्ते वेंकट! मैं कैसे मदद कर सकता हूँ?\n</output>\n\n\nअब आपकी बारी है! कृपया अपना प्रश्न पूछें।', 'executed_tools': [{'arguments': '{"code": "print("नमस्ते वेंकट! मैं कैसे मदद कर सकता हूँ?")"}', 'index': 0, 'type': 'python', 'browser_results': None, 'code_results': None, 'output': 'नमस्ते वेंकट! मैं कैसे मदद कर सकता हूँ?\n', 'search_results': {'images': None, 'results': []}}]} response_metadata={'token_usage': {'completion_tokens': 268, 'prompt_tokens': 1742, 'total_

### This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [52]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [53]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi, I am Venkat")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते वेंकट! आपसे मिलकर बहुत खुशी हुई। यदि आपके पास कोई प्रश्न है या किसी चीज़ में मदद चाहिए, तो बेझिझक बताइए—मैं यहाँ आपकी सहायता के लिए हूँ।'

In [55]:
response=with_message_history.invoke(
{'messages':[HumanMessage(content="What is my name")],"language":"Hindi"},
    config=config,
)

In [56]:
response.content

'आपने पहले बताया था कि आपका नाम **वेंकट** है।'

### Managing the Conversation History
##### One important concept to understand when building the chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the "context window" of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
##### trim_messages  helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages.

In [67]:
from langchain_core.messages import SystemMessage,trim_messages 
trimmer=trim_messages(
    max_tokens=70,          # setting max no. of token for context
    strategy="last",
    token_counter=model,
    include_system=True,    # system messages
    allow_partial=False,    # means no partial response 
    start_on="human"        # start from the human conversation
)

# here, we are setting up the LLM context 
messages=[
    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="Hi, am Venkat"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="What is 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="Its my pleasure"),
    HumanMessage(content="Having fun?"),
    AIMessage(content="Yes"),
    HumanMessage(content="What is Data Science?"),
    AIMessage(content="Data science is the practice of turning raw data into insights, predictions, and decisions using statistics, programming, and domain knowledge.")
]

trimmer.invoke(messages)

[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Its my pleasure', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is Data Science?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Data science is the practice of turning raw data into insights, predictions, and decisions using statistics, programming, and domain knowledge.', additional_kwargs={}, response_metadata={})]

In [68]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

# assigning to the variable 
response=chain.invoke(
    {"messages":messages + [HumanMessage(content="What ice cream do I like?")],
    "language":"English"
    }
)

# check the response 
response.content

'Based on the information we have so far, I don’t actually know your personal favorite ice‑cream flavor—\u200byou haven’t shared any specific details about your tastes.\u202fThe only “reasoning” I can offer right now is a general one:\n\n1. **What I know:**  \n   * You asked, “What ice cream do I like?”  \n   * I have no direct data about your past choices, dietary restrictions, or flavor preferences.\n\n2. **What I can infer:**  \n   * In the absence of personal clues, the safest approach is to look at the most‑commonly‑enjoyed flavors.\u202fThese are the flavors that many people tend to like, so there’s a reasonable chance one of them could be yours as well.\n\n3. **Popular ice‑cream flavors (the ones most people gravitate toward):**  \n   - **Vanilla** – classic, versatile, often a go‑to for toppings or desserts.  \n   - **Chocolate** – rich, indulgent, a favorite for chocolate lovers.  \n   - **Strawberry** – fruity and slightly tart, a common “fruit” choice.  \n   - **Cookies\u202

In [69]:
# Ask another question

response=chain.invoke(
    {"messages":messages + [HumanMessage(content="What Data Science question I asked?")],
    "language":"English"
    }
)

response.content

'**Your original Data‑Science‑related question was:**\n\n> **“What is Data Science?”**\n\n---\n\n### How I determined that\n\n1. **Conversation history** – Earlier you asked, “What is Data Science?” and I responded with a concise definition.\n2. **Your follow‑up** – You then asked, “What Data Science question I asked?” which is essentially a request to recall the exact wording of the previous query.\n3. **Clarification** – You later restated the request: “Remember, the initial question I asked was: What Data Science question I asked?” confirming that you want the original question identified.\n\n---\n\n### The answer (including the definition I previously gave)\n\n> **Question:** *What is Data Science?*  \n> **Answer:** Data science is the practice of turning raw data into insights, predictions, and decisions using statistics, programming, and domain knowledge. It combines techniques from mathematics, computer science, and subject‑matter expertise to extract meaningful information from

In [ ]:
# now, wrap this in MessageHistory with a new session_id
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",

)

config={"configurable":{"session_id":"chat5"}}

In [72]:
# This message history wont give the question I asked for because this is a new session_id
response=with_message_history.invoke(
    {"messages":messages + [HumanMessage(content="What's my name?")],
    "language":"English"

    },
    config=config
)

response.content

'I don’t have any information about your name from our conversation so far. If you’d like me to address you by name, could you let me know what it is?'